In [ ]:
import pandas as pd
RAW_DATA_DIR = '../data/raw'

In [ ]:
df_fifa = pd.read_csv(f'{RAW_DATA_DIR}/fifa_combined.csv', dtype= str)
df_fbref = pd.read_csv(f'{RAW_DATA_DIR}/fbref_merged_stats.csv', dtype= str)

In [ ]:
def remove_early_seasons(df):
    seasons_to_remove = ['1415', '1516', '1617']
    df_filtered = df[~df['season'].isin(seasons_to_remove)]
    return df_filtered
df_fifa = remove_early_seasons(df_fifa)
df_fifa.head(10)

In [ ]:
leagues  = df_fifa['league_name'].unique()
#leagues

In [ ]:
# Keep only FIFA leagues that correspond to FBref leagues
fifa_leagues_to_keep = [
    'Premier League',      # ENG-Premier League
    'La Liga',             # ESP-La Liga
    'Ligue 1',             # FRA-Ligue 1
    'Bundesliga',          # GER-Bundesliga
    'Serie A',             # ITA-Serie A
    'Eredivisie',          # NED-Eredivisie
    'Liga Portugal',       # POR-Primeira Liga
    'Super Lig',           # TUR-Super Lig
    'Major League Soccer', # USA-MLS
    'Jupiler Pro League',  # BEL-Jupiler Pro League
]

df_fifa = df_fifa[df_fifa['league_name'].isin(fifa_leagues_to_keep)]
df_fifa = df_fifa.dropna()
print(f"Filtered FIFA to {len(df_fifa)} rows in relevant leagues")
df_fifa.head(10)

In [ ]:
# Remove accents from all name columns, keep capitalization!
import unicodedata
def normalize_name(name):
    nfd = unicodedata.normalize('NFD', name)
    without_accents = ''.join(c for c in nfd if unicodedata.category(c) != 'Mn')
    return without_accents
df_fifa['long_name'] = df_fifa['long_name'].apply(normalize_name)
df_fifa['short_name'] = df_fifa['short_name'].apply(normalize_name)
df_fbref['player'] = df_fbref['player'].apply(normalize_name)
df_fifa.head(10)

In [ ]:
# Aggregate FBref stats for players who played at multiple clubs in same season
# Sum Playing Time columns (cumulative totals), take first for everything else (rates/averages)

# Identify Playing Time columns to sum
playing_time_cols = [col for col in df_fbref.columns if col.startswith('Playing Time_')]

# Columns to keep from first occurrence
first_cols = [col for col in df_fbref.columns if col not in ['player', 'season'] + playing_time_cols]

# Group by player+season and aggregate
agg_dict = {}
for col in playing_time_cols:
    agg_dict[col] = 'sum'  # Sum playing time totals
for col in first_cols:
    agg_dict[col] = 'first'  # Take first (primary club) for rates/averages

df_fbref_agg = df_fbref.groupby(['player', 'season'], as_index=False).agg(agg_dict)

print(f"Original FBref rows: {len(df_fbref)}")
print(f"Aggregated FBref rows: {len(df_fbref_agg)}")
print(f"Duplicate player+seasons removed: {len(df_fbref) - len(df_fbref_agg)}")
df_fbref_agg.head()


In [ ]:
import unicodedata
import re

def normalize_name(text):
    """Remove accents, lowercase, and handle punctuation/hyphens"""
    if pd.isna(text):
        return ""
    # Remove accents
    nfd = unicodedata.normalize('NFD', str(text))
    text = ''.join(c for c in nfd if unicodedata.category(c) != 'Mn')
    
    # Replace hyphens and dots with spaces so 'Heung-min' becomes 'Heung min'
    text = re.sub(r'[-.]', ' ', text)
    return text.lower().strip()

def is_latin(text):
    """Check if text contains primarily Latin characters"""
    if not text:
        return False
    # Count Latin characters (basic Latin + Latin Extended)
    latin_chars = sum(1 for c in text if ord(c) < 0x0400 and c.isalpha())
    # If more than half are Latin or it's a short word that's fully Latin, keep it
    return latin_chars > 0 and (latin_chars >= len([c for c in text if c.isalpha()]) * 0.5)

def normalize_team(team_name):
    """Normalize team names for matching"""
    if pd.isna(team_name):
        return ""
    team = normalize_name(team_name)
    # Remove common words that appear in team names
    common_words = ['fc', 'afc', 'cf', 'sc', 'united', 'city', 'real', 'athletic', 
                    'hotspur', 'wanderers', 'rovers', 'albion', 'town', 'county']
    words = team.split()
    # Keep main identifying word(s), remove common suffixes
    filtered = [w for w in words if w not in common_words]
    # If we filtered everything, keep original
    if not filtered:
        filtered = words
    return ' '.join(filtered)

def names_match(name1, name2):
    """Check if two name parts match, accounting for nicknames and partial matches"""
    if name1 == name2:
        return True
    # Check if one is a prefix of the other (e.g., Phil vs Philip)
    if name1.startswith(name2) or name2.startswith(name1):
        return True
    # Check if they share first 3+ characters (for very similar names)
    if len(name1) >= 3 and len(name2) >= 3 and name1[:3] == name2[:3]:
        return True
    return False

def teams_match(fifa_team, fbref_team):
    """Check if two teams match"""
    norm_fifa = normalize_team(fifa_team)
    norm_fbref = normalize_team(fbref_team)
    
    if not norm_fifa or not norm_fbref:
        return False
    
    # Direct match
    if norm_fifa == norm_fbref:
        return True
    
    # Check if one contains the other (word-level)
    fifa_words = set(norm_fifa.split())
    fbref_words = set(norm_fbref.split())
    
    # If there's any substantial overlap (at least one word in common)
    common_words = fifa_words & fbref_words
    if common_words:
        # Check if the common word is substantial (>3 chars)
        if any(len(w) > 3 for w in common_words):
            return True
    
    # Check string containment as fallback
    if norm_fifa in norm_fbref or norm_fbref in norm_fifa:
        return True
    
    return False

# 1. Normalize and split names (Ensures 'heung-min' -> {'heung', 'min'})
# Use df_fbref_agg (aggregated) instead of df_fbref to match what we'll merge with
df_fbref_agg['name_parts'] = df_fbref_agg['player'].apply(lambda x: set(normalize_name(x).split()))
df_fifa['long_name_lower_parts'] = df_fifa['long_name'].apply(lambda x: set(normalize_name(x).split()))
df_fifa['short_name_lower_parts'] = df_fifa['short_name'].apply(lambda x: set(normalize_name(x).split()))

# 2. Create a lookup dictionary from FBref using season and born as key
fbref_lookup = {}
for _, row in df_fbref_agg.iterrows():
    key = (row['season'], row['born'])
    if key not in fbref_lookup:
        fbref_lookup[key] = []
    fbref_lookup[key].append((row['player'], row['name_parts'], row['team']))

# 3. Optimized matching function
def match_name(row):
    key = (row['season'], row['born'])
    candidates = fbref_lookup.get(key)
    if not candidates:
        return None
    
    fifa_all_parts = row['long_name_lower_parts'] | row['short_name_lower_parts']
    # Filter out single-letter parts and non-Latin characters
    fifa_all_parts = {p for p in fifa_all_parts if len(p) > 1 and is_latin(p)}
    
    fifa_team = row.get('club_name', '')
    
    # Strategy 1: Perfect Intersection with Team Match
    # If they share at least 2 words, OR they share the only word they have
    for player_name, fb_parts, fbref_team in candidates:
        fb_parts_filtered = {p for p in fb_parts if len(p) > 1}
        
        common = fb_parts_filtered.intersection(fifa_all_parts)
        
        # Condition: If they share at least 2 words, OR they share the only word they have
        if len(common) >= 2 or (len(fb_parts_filtered) == 1 and len(common) == 1):
            # Double check with team to be safe
            if teams_match(fifa_team, fbref_team):
                return player_name
    
    # Strategy 2: Perfect Intersection without Team Match (fallback)
    for player_name, fb_parts, fbref_team in candidates:
        fb_parts_filtered = {p for p in fb_parts if len(p) > 1}
        
        common = fb_parts_filtered.intersection(fifa_all_parts)
        
        # Require at least 2 substantial matches for safety without team
        if len(common) >= 2:
            return player_name
    
    # Strategy 3: Subset Fallback with Team Match (For "H. Son" where FIFA might only have {"son"})
    for player_name, fb_parts, fbref_team in candidates:
        fb_parts_filtered = {p for p in fb_parts if len(p) > 1}
        
        if len(fifa_all_parts) > 0 and fifa_all_parts.issubset(fb_parts_filtered):
            if teams_match(fifa_team, fbref_team):
                # Require at least one substantial match (>2 chars)
                if any(len(p) > 2 for p in fifa_all_parts):
                    return player_name
    
    # Strategy 4: Reverse Subset with Team Match (FBref ⊆ FIFA)
    for player_name, fb_parts, fbref_team in candidates:
        fb_parts_filtered = {p for p in fb_parts if len(p) > 1}
        
        if len(fb_parts_filtered) > 0 and fb_parts_filtered.issubset(fifa_all_parts):
            if teams_match(fifa_team, fbref_team):
                return player_name
    
    # Strategy 5: Fuzzy name matching with Team Match (for nicknames like Phil/Philip)
    for player_name, fb_parts, fbref_team in candidates:
        if teams_match(fifa_team, fbref_team):
            fb_parts_filtered = {p for p in fb_parts if len(p) > 1}
            
            # Check if we have fuzzy matches
            fuzzy_matches = sum(1 for fp in fb_parts_filtered 
                              if any(names_match(fp, mp) for mp in fifa_all_parts))
            
            # If most parts match fuzzily
            if fuzzy_matches >= len(fb_parts_filtered) * 0.7 and fuzzy_matches > 0:
                return player_name
    
    # Strategy 6: Last resort - any fuzzy match with team (only if single name)
    for player_name, fb_parts, fbref_team in candidates:
        if teams_match(fifa_team, fbref_team):
            fb_parts_filtered = {p for p in fb_parts if len(p) > 1}
            if len(fb_parts_filtered) == 1 and any(names_match(list(fb_parts_filtered)[0], mp) for mp in fifa_all_parts):
                return player_name
    
    return None

# 4. Apply matching
df_fifa['fuzzy_name'] = df_fifa.apply(match_name, axis=1)

# Cleanup temporary columns
df_fifa.drop(columns=['long_name_lower_parts', 'short_name_lower_parts'], inplace=True)
df_fifa.head(10)

In [ ]:
fifa_cols = df_fifa.columns.tolist()
fbref_calls = df_fbref.columns.tolist()
print("FIFA columns:", fifa_cols)
print("FBref columns:", fbref_calls)

In [ ]:
# Backfill fuzzy_name: For players with missing fuzzy_name in some seasons,
# copy the fuzzy_name from other seasons of the same player (same long_name + born)

def backfill_fuzzy_names(df):
    """
    For rows where fuzzy_name is NaN, look for other rows with same long_name and born
    that have a fuzzy_name, and copy it.
    
    Args:
        df: DataFrame with fuzzy_name column that may have NaN values
    
    Returns:
        Updated dataframe with backfilled fuzzy_name values
    """
    df = df.copy()
    
    # Find rows with missing fuzzy_name
    missing_mask = df['fuzzy_name'].isnull()
    print(f"Rows with missing fuzzy_name: {missing_mask.sum()}")
    
    if missing_mask.sum() == 0:
        print("No missing fuzzy_name values to backfill")
        return df
    
    # Create lookup: (long_name, born) -> fuzzy_name
    # Use the first non-null fuzzy_name for each player
    player_fuzzy_map = {}
    for _, row in df[df['fuzzy_name'].notnull()].iterrows():
        key = (row['long_name'], row['born'])
        if key not in player_fuzzy_map:
            player_fuzzy_map[key] = row['fuzzy_name']
    
    # Fill missing fuzzy_name values
    filled_count = 0
    for idx in df[missing_mask].index:
        row = df.loc[idx]
        key = (row['long_name'], row['born'])
        
        if key in player_fuzzy_map:
            df.at[idx, 'fuzzy_name'] = player_fuzzy_map[key]
            filled_count += 1
    
    print(f"Backfilled {filled_count} fuzzy_name values from other seasons")
    print(f"Remaining missing: {df['fuzzy_name'].isnull().sum()}")
    return df

print("Backfill function created. Usage:")
print("df_fifa = backfill_fuzzy_names(df_fifa)")

In [ ]:
# Apply the backfill
df_fifa = backfill_fuzzy_names(df_fifa)

In [ ]:
# Remove no matches
df_fifa = df_fifa[df_fifa['fuzzy_name'].notnull()]
print(f"After removing unmatched, FIFA has {len(df_fifa)} rows")

In [ ]:
# Merge FIFA with aggregated FBref stats
df_merged = df_fifa.merge(
    df_fbref_agg, 
    left_on=['fuzzy_name', 'season'], 
    right_on=['player', 'season'], 
    how='inner',  # Use inner since we removed unmatched
    suffixes=('_fifa', '_fbref')
)

print(f"FIFA rows: {len(df_fifa)}")
print(f"Merged rows: {len(df_merged)}")
print(f"Columns: {len(df_merged.columns)}")

# Verify no missing FBref data
missing_fbref = df_merged['player'].isnull().sum()
if missing_fbref > 0:
    print(f"⚠️  Warning: {missing_fbref} rows missing FBref data after merge")
else:
    print("✅ All rows have FBref data")

df_merged.head()

In [ ]:
# Drop duplicate column (fuzzy_name = player after merge)
df_merged = df_merged.drop(columns=['fuzzy_name'])

#df_merged.to_csv('../data/clean/fifa_fbref_merged.csv', index=False)
#print(f"✅ Saved {len(df_merged)} rows to fifa_fbref_merged.csv")
df_merged

In [ ]:
#show where 'pos' is nan
df_null = df_merged[df_merged['pos'].isnull()]
df_null.head()